### Deep Kung-Fu with advantage actor-critic

In this notebook you'll build a deep reinforcement learning agent for Atari [Kung-Fu Master](https://gym.openai.com/envs/KungFuMaster-v0/) and train it with Advantage Actor-Critic.

Note that, strictly speaking, this will be neither [A3C](https://arxiv.org/abs/1602.01783) nor [A2C](https://openai.com/blog/baselines-acktr-a2c/), but rather a simplified version of the latter.

Special thanks to Jesse Grabowski for making an [initial version](https://www.coursera.org/learn/practical-rl/discussions/all/threads/6iDjkbhPQoGg45G4T7KBHQ/replies/5eM_hA7vEeuoKgpcLmLqdw) of the PyTorch port of this assignment.

![https://upload.wikimedia.org/wikipedia/en/6/66/Kung_fu_master_mame.png](https://upload.wikimedia.org/wikipedia/en/6/66/Kung_fu_master_mame.png)

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    %tensorflow_version 1.x
    
    if not os.path.exists('.setup_complete'):
        !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash

        !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/grading.py -O ../grading.py
        !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week5_policy_based/submit.py
        !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week5_policy_based/atari_util.py

        !touch .setup_complete

# If you are running on a server, launch xvfb to record game videos
# Please make sure you have xvfb installed
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display

For starters, let's take a look at the game itself:

* Image resized to 42x42 and converted to grayscale to run faster
* Agent sees last 4 frames of game to account for object velocity

In [ ]:
import gym
from atari_util import PreprocessAtari

def make_env():
    env = gym.make("KungFuMasterDeterministic-v0")
    env = PreprocessAtari(
        env, height=42, width=42,
        crop=lambda img: img[60:-30, 5:],
        dim_order='pytorch',
        color=False, n_frames=4)
    return env

env = make_env()

obs_shape = env.observation_space.shape
n_actions = env.action_space.n

print("Observation shape:", obs_shape)
print("Num actions:", n_actions)
print("Action names:", env.env.env.get_action_meanings())

In [ ]:
s = env.reset()
for _ in range(100):
    s, _, _, _ = env.step(env.action_space.sample())

plt.title('Game image')
plt.imshow(env.render('rgb_array'))
plt.show()

plt.title('Agent observation (4-frame buffer)')
plt.imshow(s.transpose([1, 0, 2]).reshape([42, -1]), cmap='gray')
plt.show()

### Build a network

We now have to build an agent for actor-critic training — a convolutional neural network that converts states into action probabilities $\pi$ and state values $V$.

Your assignment here is to build and apply a neural network. You can use any framework you want, but in this notebook we prepared for you a template in PyTorch.

For starters, we want you to implement this architecture:

![https://s17.postimg.cc/orswlfzcv/nnet_arch.png](https://s17.postimg.cc/orswlfzcv/nnet_arch.png)

Notes:
* This diagram was originally made for Tensorflow. In PyTorch, the input shape is `[batch_size, 4, 42, 42]`.
* Use convolution kernel size 3x3 throughout.
* After your agent gets mean reward above 5000, we encourage you to experiment with model architecture to score even better.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
def conv2d_size_out(size, kernel_size, stride):
    """
    Helper function to compute the spatial dimensions of the output
    of a convolutional layer, copied from Week 4.

    Common use case:
        cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
        cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    This can be used to understand the shape for dense layer's input.
    """
    return (size - (kernel_size - 1) - 1) // stride + 1

In [ ]:
class Agent(nn.Module):
    def __init__(self, input_dims, n_actions, lr):
        super(Agent, self).__init__()
        
        self.input_dims = input_dims

        # Initialize layers as shown in image above
        <YOUR CODE>

        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        self.device = device
        self.to(self.device)
        
    def forward(self, state):
        # Compute logits and values using network.
        # Note that if you do so naively, your state_values will have shape
        # ending in 1, since they come from a Linear(..., 1) layer. It is useful
        # to .squeeze(dim=-1) them, since this will help avoid shape conflicts
        # in the loss function part, after we add multiple environments.
        # If you don't do this here, don't forget to do that in the
        # loss function!
        
        <YOUR CODE>

        return logits, state_values

    def choose_action(self, observation):
        # PyTorch wants a batch dimension, so if we feed only a single observation we need to wrap it with an extra layer.
        # This line will allow the network to handle both single and multi-environment tests.
        if observation.ndim == 3:
            observation = [observation]

        observation = torch.tensor(observation, dtype=torch.float32, device=device)
        # Pass states into agent network and get back logits and states
        logits, _ = <YOUR CODE>
                
        policy = F.softmax(logits, dim=-1)

        actions = np.array([np.random.choice(len(p), p=p) for p in policy.detach().cpu().numpy()])
        return actions

In [ ]:
# Test network

agent = Agent(input_dims=obs_shape, n_actions=n_actions, lr=1e-4)
state = env.reset()
state = torch.tensor([state], dtype=torch.float32, device=device)
logits, state_values = agent(state)

assert isinstance(logits, torch.Tensor) and len(logits.shape) == 2, \
    "Please return a 2D Torch tensor of logits with shape (batch_size, n_actions). You returned %s" % repr(logits)
assert isinstance(state_values, torch.Tensor) and len(state_values.shape) == 1, \
    "Please return a 1D Torch tensor of state values with shape (batch_size,). You returned %s" % repr(state_values)

###Actor-Critic

Here we define loss functions and learning algorithms as usual.

In [ ]:
def compute_actor_critic_loss(agent, state, action, reward, next_state, done,
                              gamma=0.99):
    # Infer batch_size from shape of state tensor:
    batch_size = state.shape[0]

    # Convert everything to a tensor, send to GPU if available
    state      = torch.tensor(state, dtype=torch.float32, device=device)
    next_state = torch.tensor(next_state, dtype=torch.float32, device=device)
    reward     = torch.tensor(reward, dtype=torch.float32, device=device)
    done       = torch.tensor(done, dtype=torch.bool, device=device)

    # logits[n_envs, n_actions] and state_values[n_envs, n_actions]
    logits, state_value = agent(state)
    next_logits, next_state_value = agent(next_state)

    # Probabilities and log-probabilities for all actions
    probs    = F.softmax(logits, dim=-1)     #[n_envs, n_actions]
    logprobs = F.log_softmax(logits, dim=-1) #[n_envs, n_actions]

    # Set new state values with done == 1 to be 0.0 (no future rewards if done!)
    next_state_value[done] = 0.0

    # Compute target state values using temporal difference formula.
    # Use reward, gamma, and next_state_value.
    target_state_value = <YOUR CODE>

    # Compute advantage using reward, gamma, state_value, and next_state_value.
    advantage = <YOUR CODE>

    # We need to slice out only the actions we took for actor loss -- we can use
    # the actions themselves as indexes, but we also need indexes on the batch dim
    batch_idx = np.arange(batch_size)
    logp_actions = logprobs[batch_idx, action]

    # Compute policy entropy given logits_seq. Mind the "-" sign!
    entropy = <YOUR CODE>

    actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()
    critic_loss = F.mse_loss(target_state_value.detach(), state_value)

    total_loss = actor_loss + critic_loss

    # Never forget to zero grads in PyTorch!
    agent.optimizer.zero_grad()
    total_loss.backward()
    agent.optimizer.step()

    return actor_loss.cpu().detach().numpy(), critic_loss.cpu().detach().numpy(), entropy.cpu().detach().numpy()

In [ ]:
state = env.reset()
state = torch.tensor([state], dtype=torch.float32).to(device)
logits, value = agent(state)
print("action logits:\n", logits)
print("state values:\n", value)

### Let's play!
Let's build a function that measures the agent's average reward.

In [ ]:
def evaluate(agent, env, n_games=1):
    """Plays an a game from start till done, returns per-game rewards """

    game_rewards = []
    for _ in range(n_games):
        state = env.reset()

        total_reward = 0
        while True:
            action = agent.choose_action(state)
            state, reward, done, info = env.step(action)
            total_reward += reward
            if done:
                break

        game_rewards.append(total_reward)
    return game_rewards

In [ ]:
import gym.wrappers

with gym.wrappers.Monitor(make_env(), directory="videos", force=True) as env_monitor:
    rewards = evaluate(agent, env_monitor, n_games=3)

print(rewards)

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices

### Training on parallel games

![img](https://s7.postimg.cc/4y36s2b2z/env_pool.png)


To make actor-critic training more stable, we shall play several games in parallel. This means ya'll have to initialize several parallel gym envs, send agent's actions there and .reset() each env if it becomes terminated. To minimize learner brain damage, we've taken care of them for ya - just make sure you read it before you use it.

In [ ]:
class EnvBatch:
    def __init__(self, n_envs = 10):
        """ Creates n_envs environments and babysits them for ya' """
        self.envs = [make_env() for _ in range(n_envs)]
        
    def reset(self):
        """ Reset all games and return [n_envs, *obs_shape] observations """
        return np.array([env.reset() for env in self.envs])
    
    def step(self, actions):
        """
        Send a vector[batch_size] of actions into respective environments
        :returns: observations[n_envs, *obs_shape], rewards[n_envs], done[n_envs,], info[n_envs]
        """
        results = [env.step(a) for env, a in zip(self.envs, actions)]
        new_obs, rewards, done, infos = map(np.array, zip(*results))
        
        # reset environments automatically
        for i in range(len(self.envs)):
            if done[i]:
                new_obs[i] = self.envs[i].reset()
        
        return new_obs, rewards, done, infos

__Let's try it out:__

In [ ]:
env_batch = EnvBatch(10)

batch_states = env_batch.reset()
batch_actions = agent.choose_action(batch_states)
batch_next_states, batch_rewards, batch_done, _ = env_batch.step(batch_actions)

print("State shape:", batch_states.shape)
print("Actions:", batch_actions)
print("Rewards:", batch_rewards)
print("Done:", batch_done)

# Sanity Check

In [ ]:
agent = Agent(lr=1e-4, n_actions=n_actions, input_dims=obs_shape)

state = env_batch.reset()
action = agent.choose_action(state)

next_state, reward, done, info = env_batch.step(action)

l_act, l_crit, ent = compute_actor_critic_loss(agent, state, action, reward, next_state, done)

assert abs(l_act) < 100 and abs(l_crit) < 100, "losses seem abnormally large"
assert 0 <= ent.mean() <= np.log(n_actions), "impossible entropy value, double-check the formula pls"
if ent.mean() < np.log(n_actions) / 2:
    print("Entropy is too low for an untrained agent")
print("You just might be fine!")

# Train 

Just the usual - play a bit, compute loss, follow the graidents, repeat a few million times.

![Daniel San training the Karate Kid](https://media.giphy.com/media/W4uQMqlKVoiXK89T5j/giphy.gif)

In [ ]:
import pandas as pd

def ewma(x, span=100):
    return pd.DataFrame({'x':np.asarray(x)}).x.ewm(span=span).mean().values

env_batch = EnvBatch(10)
batch_states = env_batch.reset()

rewards_history = []
entropy_history = []

Please pay extra attention to how we scale rewards in training. We do that for multiple reasons.

1. All rewards are multiples of 100, and even an untrained agent can get a score of 800. Therefore, even in the beginning of training, the critic will have to predict pretty large numbers. Neural networks require extra tinkering to output large numbers reliably. In this case, the easiest workaround is just to scale back those numbers.
2. We have already tweaked the hyperparameters (loss coefficients) to work well with this scaling.

Please note however that we would not have needed to do this in plain REINFORCE without entropy regularization but with Adam optimizer.

In REINFORCE, there is only actor and no critic. Without entropy regularization, actor loss is just policy gradient. It is proportional to rewards, but it only affects the scale of the gradient. However, Adam maintains a running average of the variance of the gradient for each parameter it optimizes, and normalizes the gradient by its variance in each optimization step. This will negate any scaling of the gradient.

If your implementation works correctly, you can comment out the `batch_rewards = batch_rewards * 0.01` line, restart training, and see it explode.

In [ ]:
import tqdm
from IPython.display import clear_output

with tqdm.trange(len(entropy_history), 100000) as progress_bar:
    for i in progress_bar:
        batch_actions = agent.choose_action(batch_states)
        batch_next_states, batch_rewards, batch_done, _ = env_batch.step(batch_actions)

        # Reward scaling. See above for explanation.
        batch_rewards = batch_rewards * 0.01

        agent_loss, critic_loss, entropy = compute_actor_critic_loss(
            agent, batch_states, batch_actions, batch_rewards, batch_next_states, batch_done)
        entropy_history.append(np.mean(entropy))

        batch_states = batch_next_states

        if i % 500 == 0:
            if i % 2500 == 0:
                rewards_history.append(np.mean(evaluate(agent, env, n_games=3)))
                if rewards_history[-1] >= 5000:
                    print("Your agent has earned the yellow belt")

            clear_output(True)

            plt.figure(figsize=[8, 4])
            plt.subplot(1, 2, 1)
            plt.plot(rewards_history, label='rewards')
            plt.plot(ewma(np.array(rewards_history), span=10), marker='.', label='rewards ewma@10')
            plt.title("Session rewards")
            plt.grid()
            plt.legend()

            plt.subplot(1, 2, 2)
            plt.plot(entropy_history, label='entropy')
            plt.plot(ewma(np.array(entropy_history), span=1000), marker='.', label='entropy ewma@1000')
            plt.title("Policy entropy")
            plt.grid()
            plt.legend()
            plt.show()

Relax and grab some refreshments while your agent is locked in an infinite loop of violence and death.

__How to interpret plots:__

The session reward is the easy thing: it should in general go up over time, but it's okay if it fluctuates ~~like crazy~~. It's also OK if it doesn't increase substantially before some 10-20k initial steps, and some people who tried this assignment [told us](https://www.coursera.org/learn/practical-rl/discussions/all/threads/3OnFNVxEEemLZA644RFX2A) they didn't see improvements until around 60k steps. However, if reward reaches zero and doesn't seem to get up over 2-3 evaluations, there's something wrong happening.

Since we use a policy-based method, we also keep track of __policy entropy__ — the same one you used as a regularizer. The only important thing about it is that your entropy shouldn't drop too low (`< 0.1`) before your agent gets the yellow belt. Or at least it can drop there, but _it shouldn't stay there for long_.

If it does, the culprit is likely:
* Some bug in entropy computation. Remember that it is $-\sum p(a_i) \cdot \log p(a_i)$.
* Your model architecture is broken in some way: for example, if you create layers in `Agent.symbolic_step()` rather than in `Agent.__init__()`, then effectively you will be training two separate agents: one for `logits, state_values` and another one for `next_logits, next_state_values`.
* Your architecture is different from the one we suggest and it converges too quickly. Change your architecture or increase entropy coefficient in actor loss. 
* Gradient explosion: just [clip gradients](https://stackoverflow.com/a/43486487) and maybe use a smaller network
* Us. Or TF developers. Or aliens. Or lizardfolk. Contact us on forums before it's too late!

If you're debugging, just run `logits, values = agent.step(batch_states)` and manually look into logits and values. This will reveal the problem 9 times out of 10: you'll likely see some NaNs or insanely large numbers or zeros. Try to catch the moment when this happens for the first time and investigate from there.

### "Final" evaluation

In [ ]:
import gym.wrappers

with gym.wrappers.Monitor(make_env(), directory="videos", force=True) as env_monitor:
    final_rewards = evaluate(agent, env_monitor, n_games=3)

print("Final mean reward:", np.mean(final_rewards))

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))

In [ ]:
HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-2]))  # You can also try other indices

If you don't see videos above, just navigate to `./videos` and download `.mp4` files from there.

In [ ]:
from submit import submit_kungfu
env = make_env()
submit_kungfu(agent, env, evaluate, 'your.email@example.com', 'YourAssignmentToken')

### Now what?
Well, 5k reward is [just the beginning](https://www.buzzfeed.com/mattjayyoung/what-the-color-of-your-karate-belt-actually-means-lg3g). Can you get past 200? With recurrent neural network memory, chances are you can even beat 400!

* Try n-step advantage and "lambda"-advantage (aka GAE) - see [this article](https://arxiv.org/abs/1506.02438)
 * This change should improve early convergence a lot
* Try recurrent neural network 
 * RNN memory will slow things down initially, but in will reach better final reward at this game
* Implement asynchronuous version
 * Remember [A3C](https://arxiv.org/abs/1602.01783)? The first "A" stands for asynchronuous. It means there are several parallel actor-learners out there.
 * You can write custom code for synchronization, but we recommend using [redis](https://redis.io/)
   * You can store full parameter set in redis, along with any other metadate
   * Here's a _quick_ way to (de)serialize parameters for redis
   ```
   import joblib
   from six import BytesIO
```
```
   def dumps(data):
        "converts whatever to string"
        s = BytesIO()
        joblib.dump(data,s)
        return s.getvalue()
``` 
```
    def loads(string):
        "converts string to whatever was dumps'ed in it"
        return joblib.load(BytesIO(string))
```